In [45]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from datetime import datetime, timedelta
import time
import random
from modules.peers_view import PeersView
from modules.user import User

In [36]:
addresses = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
N = 20
EXPIRATION_PERIOD = timedelta(seconds=1)
print("Expiration period:", EXPIRATION_PERIOD)

address_book = PeersView(expiration_period=EXPIRATION_PERIOD)

for _ in range(N):
    connected = random.sample(addresses, k=random.randint(1, int(2 * len(addresses) / 3)))
    print("Time {}: connected = {}".format(datetime.now().time(), connected))
    
    for d in connected:
        address_book.put(datetime.now(), d, 0.8)
        time.sleep(0.1)
    
    print("Address book:")
    print(address_book.view)
    print()

Expiration period: 0:00:01
Time 15:55:41.756095: connected = ['a', 'b']
Address book:
                           addr    p                          t
2018-04-16 15:55:41.756697    a  0.8 2018-04-16 15:55:41.756697
2018-04-16 15:55:41.869579    b  0.8 2018-04-16 15:55:41.869579

Time 15:55:41.994660: connected = ['f']
Address book:
                           addr    p                          t
2018-04-16 15:55:41.756697    a  0.8 2018-04-16 15:55:41.756697
2018-04-16 15:55:41.869579    b  0.8 2018-04-16 15:55:41.869579
2018-04-16 15:55:41.994783    f  0.8 2018-04-16 15:55:41.994783

Time 15:55:42.105206: connected = ['c', 'a', 'f', 'g', 'i']
Address book:
                           addr    p                          t
2018-04-16 15:55:41.869579    b  0.8 2018-04-16 15:55:41.869579
2018-04-16 15:55:42.105265    c  0.8 2018-04-16 15:55:42.105265
2018-04-16 15:55:42.208500    a  0.8 2018-04-16 15:55:42.208500
2018-04-16 15:55:42.312630    f  0.8 2018-04-16 15:55:42.312630
2018-04-16 15:55

Address book:
                           addr    p                          t
2018-04-16 15:55:49.588523    b  0.8 2018-04-16 15:55:49.588523
2018-04-16 15:55:49.717332    e  0.8 2018-04-16 15:55:49.717332
2018-04-16 15:55:49.843484    h  0.8 2018-04-16 15:55:49.843484
2018-04-16 15:55:49.957310    j  0.8 2018-04-16 15:55:49.957310
2018-04-16 15:55:50.063513    g  0.8 2018-04-16 15:55:50.063513
2018-04-16 15:55:50.177440    f  0.8 2018-04-16 15:55:50.177440
2018-04-16 15:55:50.294685    d  0.8 2018-04-16 15:55:50.294685
2018-04-16 15:55:50.411149    c  0.8 2018-04-16 15:55:50.411149

Time 15:55:50.540221: connected = ['f', 'b', 'c']
Address book:
                           addr    p                          t
2018-04-16 15:55:49.843484    h  0.8 2018-04-16 15:55:49.843484
2018-04-16 15:55:49.957310    j  0.8 2018-04-16 15:55:49.957310
2018-04-16 15:55:50.063513    g  0.8 2018-04-16 15:55:50.063513
2018-04-16 15:55:50.294685    d  0.8 2018-04-16 15:55:50.294685
2018-04-16 15:55:50.54042

# Several users

In [74]:
N_USERS = 3
PERIOD = timedelta(seconds=1)
N_ROUNDS = 10

address_book = PeersView(expiration_period=PERIOD)

users = [None] * N_USERS
for i in range(N_USERS):
    users[i] = User(address_book)
    
    print("User #{} has {} devices: {}".format(
        i+1, users[i].n_devices,
        [d.addr for d in users[i].devices]))

print()

for t in range(N_ROUNDS):
    print("Round #{} at {}:".format(t+1, datetime.now().time()))
    for i in range(N_USERS):
        users[i].act()
        
        online_devices = [d.addr for d in users[i].devices if d.is_online]
        print("User#{}'s online devices: {}".format(i+1, online_devices))
    
    print("Address book:")
    print(address_book.view)
    print()
    time.sleep(PERIOD.total_seconds())

User #1 has 8 devices: ['6c9m4wjycc60', 'x2w776d4c117', 'lmuos0fpxbkq', 'tii7xm2bu4of', 'dhurpjqjkaev', 'elcsjv657mbm', 'mr1p90yfn0wc', 'y6xswkedlvkj']
User #2 has 3 devices: ['cp0ex118ri9s', '8kj2c0m1k5m3', 'uhov8saa4sn5']
User #3 has 4 devices: ['z6t7ogku1mxd', 'kz1hgnxrz1u7', 'enb4bu3alvgw', 's7fwcdee5btv']

Round #1 at 17:54:27.622784:
User#1's online devices: ['6c9m4wjycc60', 'x2w776d4c117', 'dhurpjqjkaev', 'y6xswkedlvkj']
User#2's online devices: ['cp0ex118ri9s', '8kj2c0m1k5m3']
User#3's online devices: ['z6t7ogku1mxd', 'enb4bu3alvgw', 's7fwcdee5btv']
Address book:
                                    addr         p                          t
2018-04-16 17:54:27.623439  6c9m4wjycc60  0.938090 2018-04-16 17:54:27.623439
2018-04-16 17:54:27.627594  x2w776d4c117  0.558142 2018-04-16 17:54:27.627594
2018-04-16 17:54:27.635361  dhurpjqjkaev  0.427686 2018-04-16 17:54:27.635361
2018-04-16 17:54:27.645938  y6xswkedlvkj  0.792730 2018-04-16 17:54:27.645938
2018-04-16 17:54:27.657679  cp0e

Round #10 at 17:54:37.943639:
User#1's online devices: ['6c9m4wjycc60', 'dhurpjqjkaev', 'y6xswkedlvkj']
User#2's online devices: ['cp0ex118ri9s']
User#3's online devices: ['z6t7ogku1mxd', 'enb4bu3alvgw', 's7fwcdee5btv']
Address book:
                                    addr         p                          t
2018-04-16 17:54:37.946849  6c9m4wjycc60  0.931840 2018-04-16 17:54:37.946849
2018-04-16 17:54:37.965472  dhurpjqjkaev  0.508850 2018-04-16 17:54:37.965472
2018-04-16 17:54:37.984593  y6xswkedlvkj  0.798402 2018-04-16 17:54:37.984593
2018-04-16 17:54:37.996004  cp0ex118ri9s  0.818268 2018-04-16 17:54:37.996004
2018-04-16 17:54:38.008484  z6t7ogku1mxd  0.828163 2018-04-16 17:54:38.008484
2018-04-16 17:54:38.020385  enb4bu3alvgw  0.809786 2018-04-16 17:54:38.020385
2018-04-16 17:54:38.034217  s7fwcdee5btv  0.449114 2018-04-16 17:54:38.034217

